In [0]:
import pandas as pd
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("MyApp").getOrCreate()

# Specify your target container, parent folder, and the "raw" folder
container_name = "publictransportdata"
parent_folder = "public_transport_data"
folder_name = "raw"

# Loop through the desired months (January to May)
for month in range(1, 6):
    # Generate data for the specified month
    start_date = datetime(2023, month, 1)
    if month == 5:
        end_date = datetime(2023, 5, 31)  # May has 31 days
    else:
        end_date = datetime(2023, month + 1, 1) - timedelta(days=1)

    date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date-start_date).days + 1)]

    transport_types = ["Bus", "Train", "Tram", "Metro"]
    routes = ["Route_" + str(i) for i in range(1, 11)]
    stations = ["Station_" + str(i) for i in range(1, 21)]

    # Randomly select 5 days as extreme weather days
    extreme_weather_days = random.sample(date_generated, 5)

    data = []

    for date in date_generated:
        for _ in range(32):  # 32 records per day to get a total of 992 records per month
            transport = random.choice(transport_types)
            route = random.choice(routes)

            # Normal operating hours
            departure_hour = random.randint(5, 22)
            departure_minute = random.randint(0, 59)

            # Introducing Unusual Operating Hours for buses
            if transport == "Bus" and random.random() < 0.05:  # 5% chance
                departure_hour = 3

            departure_time = f"{departure_hour:02}:{departure_minute:02}"

            # Normal duration
            duration = random.randint(10, 120)

            # Introducing Short Turnarounds
            if random.random() < 0.05:  # 5% chance
                duration = random.randint(1, 5)

            # General delay
            delay = random.randint(0, 15)

            # Weather Impact
            if date in extreme_weather_days:
                # Increase delay by 10 to 60 minutes
                delay += random.randint(10, 60)

                # 10% chance to change the route
                if random.random() < 0.10:
                    route = random.choice(routes)

            total_minutes = departure_minute + duration + delay
            arrival_hour = departure_hour + total_minutes // 60
            arrival_minute = total_minutes % 60
            arrival_time = f"{arrival_hour:02}:{arrival_minute:02}"

            passengers = random.randint(1, 100)
            departure_station = random.choice(stations)
            arrival_station = random.choice(stations)

            data.append([date, transport, route, departure_time, arrival_time, passengers, departure_station, arrival_station, delay])

    df = pd.DataFrame(data, columns=["Date", "TransportType", "Route", "DepartureTime", "ArrivalTime", "Passengers", "DepartureStation", "ArrivalStation", "Delay"])
    
    # Convert the pandas DataFrame to a PySpark DataFrame
    spark_df = spark.createDataFrame(df)

    # Specify the path in Azure Data Lake Storage Gen2
    data_path = f"abfss://{container_name}@faissalmoufllastorage.dfs.core.windows.net/{parent_folder}/{folder_name}/public_transport_data_{month}.csv"

    # Write the PySpark DataFrame to the "raw" folder in ADLS Gen2 as a CSV file
    spark_df.write.csv(data_path, mode="overwrite", header=True, sep=",")
